# ArcPy Lab 1 davisdan


#### Start by setting up arcpy

In [1]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

#### Setting workspace so that I know where all files will be placed for easy access.

In [2]:
#important to set workspace so all files are directed to my working directory
arcpy.env.workspace = "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\Workspace.gdb"

#### Allow edits made to overwrite and make a copy of original data so that I keep the original file the same.

In [3]:
arcpy.env.overwriteOutput = True

#copy features for redundancy to keep original data seperate.
arcpy.management.CopyFeatures(
    "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\saep_bg10.shp",
    "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\saep_bg10_copy.shp")

<Result 'C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\saep_bg10_copy.shp'>

#### variables to make the code easily adjusted for working with different data if given

In [4]:
fc = "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\saep_bg10_copy.shp" 
#store as a variable for speed and accuracy when running code below

#### Joining the two tables to allow county names to be listed in the main shapefile

In [5]:
#more variables to make this easier to adjust if ever given data from another source such as Oregon.
jt = "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\WashingtonFIPS.dbf"
arcpy.JoinField_management (fc, "COUNTYFP10", jt, "FIPSCounty", ["CountyName"])
#join the table and the shapefile  with matching fileds and create a new field named CountyName
#This was a handy step that I learned to do from arc help online. 
#Made it easy to create a new field through this process utilizing the syntax from ArcHelp online.

<Result 'C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\saep_bg10_copy.shp'>

#### Create a list of the counties so that I can then call back to the list when creating individual shapefiles for each specific county

In [10]:
#creating a county list for my loop to go through the shapefile and know what to search for
county_list = []

cursor = arcpy.da.SearchCursor (jt, ["CountyName"])

for row in cursor:
    county_list.append(row[0])

del cursor
print county_list

[u'Adams', u'Asotin', u'Benton', u'Chelan', u'Clallam', u'Clark', u'Columbia', u'Cowlitz', u'Douglas', u'Ferry', u'Franklin', u'Garfield', u'Grant', u'Grays Harbor', u'Island', u'Jefferson', u'King', u'Kitsap', u'Kittitas', u'Klickitat', u'Lewis', u'Lincoln', u'Mason', u'Okanogan', u'Pacific', u'Pend Oreille', u'Pierce', u'San Juan', u'Skagit', u'Skamania', u'Snohomish', u'Spokane', u'Stevens', u'Thurston', u'Wahkiakum', u'Walla Walla', u'Whatcom', u'Whitman', u'Yakima']


#### Now to run the code that will actually use the list above to create each individual shapefile of only block groups located in that county

In [11]:
work = "C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\"
# creating a loop to allow each county to be given its own shapefile 
# with the needed data from the original shapefile that I joined together above.
for county in county_list: 
    arcpy.FeatureClassToFeatureClass_conversion (fc, work, str(county), 
                                                 '"CountyName" = ' + "'%s'" %county)
    #found this toolset on Arc help Online which took a little bit to teach myself 
    #how to understand the operation and what to use as the expression to get it to do the operation successfully
    #I messed something up at this point the first time hardcore and created a lock
    #I could not figure out how to fix this so I had to start over and re-run all the code again 
    #in a new folder as it is still suffering from a lock on terminal server.

#### The code given to us in lab document that will allow shapefiles to be turned into geojson files

In [12]:
#all of this is called from the lab document given to us.
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"


#### Code that will allow the shapefiles to be turned into geojson files, with a coordinate transformation which is required for viewing in that program

In [15]:
# also from the lab and followed the steps posted to figure out what projection to use
for county in county_list:
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927', #adjusting the projection from original saep_bg10.prj to 2927 - NAD_1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet
      'C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\geojson\\' + county + '.geojson',
      'C:\\Users\\davisdan\\Documents\\geog458python\\arcpy_lab_1\\' + county + '.shp'])
    #add this to a new folder \\geojson\\ so that it is seperate and orderly in its own destination
    
    #dont forget to turn in this folder containing the geojson files for lab 1

#### Combine the population of each county to then be sorted


In [32]:
# need to create a list of population with loops and print it in descending order. county name does not need to be included
pop17_list = []
for county in county_list:
    pop17 = 0      #need use of search cursor to go through and find the needed values
    cursor = arcpy.da.SearchCursor(fc, ["POP2017"], '"CountyName" = ' + "'%s'" % county)
    for row in cursor:
        pop17 = row[0] + pop17
    pop17_list.append(pop17) 
    #this printed all data values so now i need to add in more code.
    #remember where things are formated. I had pop17_list.append(pop17) spaced incorrectly
    #it printed all rows instead of combining each county...
        

del cursor #dont forget to delete the cursor!
print pop17_list

[19870.0, 22290.0, 193499.99999999997, 76829.99999999999, 74240.00000000001, 470999.9999999994, 4100.0, 105900.00000000001, 41420.0, 7740.0, 90330.00000000003, 2200.0, 95630.0, 72970.0, 82790.0, 31360.0, 2153700.0, 264300.00000000006, 44730.0, 21660.0, 77440.00000000003, 10700.0, 63190.00000000001, 42110.00000000001, 21249.999999999996, 13370.0, 859400.0000000003, 16510.0, 124100.00000000001, 11690.0, 789400.0000000005, 499800.0, 44510.0, 276899.9999999999, 4030.0, 61399.999999999985, 216300.00000000003, 48639.99999999999, 253000.0]


#### Now that I have the population of the counties I need to sort them  to be the top 10 & in descending order

In [37]:
pop17_list.sort(reverse=True) #allows the list of the population to be sorted in a descending format
# need to now only include the top ten county populations
for i in range (10): #This tells it to only include the 10 values needed
    print pop17_list[i]

2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


#### Final thoughts and reflections

This lab was very intimidating at first but all the in class materials and group excercises helped in making this whole lab come together.
Also other students that had prior coding experience were able to break things down step by step, to create myself a workflow of what I am trying to accomplish, which made it a lot easier to think of doing little tasks line by line which then adds to the end goal.
Thank you Yulong for breaking down some of these componants to help teach me a process that I understand and can use for future workflows of solving how to code properly.
I need to remember that certain cell blocks should not be ran a second time once successfully compleated as it can cause issues like I had before.

Using variables were super helpful and I hope to use them more often as it makes it easier and looks clean when following along.

One thing that was helpful was the ArcHelp Online which had a lot of syntax examples for the toolsets so that I could understand what each operation was doing and what parameters were needed to run them properly.

#### overall I did 3 seperate days of working which combined to about 11 hours with finally success!!!